In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import math
import numpy as np

In [5]:
from typing import List
from bs4 import BeautifulSoup
import requests

In [6]:
def scrape_country() -> List:
    """Scrapes ticker symbols of top 100 US companies (based on market cap)
    Parameters
    ----------
    num_pages: int
        number of pages to scrape, each page has 100 symbols, max is 
    
    Returns
    -------
        list of strings: 100 ticker symbols
    """
    
    url = "https://companiesmarketcap.com/"
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    return [e.text for e in soup.select('span.responsive-hidden') if e ]

In [15]:
scrape_country()

['\n\n\n',
 'USA',
 'USA',
 'S. Arabia',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'Taiwan',
 'USA',
 'China',
 'USA',
 'USA',
 'USA',
 'USA',
 'S. Korea',
 'France',
 'USA',
 'USA',
 'China',
 'USA',
 'USA',
 'Switzerland',
 'USA',
 'China',
 'Switzerland',
 'USA',
 'USA',
 'Netherlands',
 'Japan',
 'USA',
 'USA',
 'China',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'France',
 'USA',
 'USA',
 'USA',
 'India',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'Denmark',
 'Ireland',
 'USA',
 'USA',
 'USA',
 'China',
 'China',
 'USA',
 'USA',
 'USA',
 'Switzerland',
 'USA',
 'Netherlands',
 'India',
 'China',
 'USA',
 'USA',
 'UK',
 'China',
 'USA',
 'USA',
 'Netherlands',
 'USA',
 'Australia',
 'France',
 'Canada',
 'UK',
 'USA',
 'Germany',
 'China',
 'USA',
 'USA',
 'USA',
 'USA',
 'China',
 'USA',
 'France',
 'Canada',
 'USA',
 'China',
 'USA',
 'USA',
 'France',
 'Ireland',
 'Japan',
 'UK',
 'Hong Kong',
 'USA',
 'Germany',
 'China']

In [8]:
def scrape_names() -> List:
    """Scrapes ticker symbols of top 100 US companies (based on market cap)
    Parameters
    ----------
    num_pages: int
        number of pages to scrape, each page has 100 symbols, max is 
    
    Returns
    -------
        list of strings: 100 ticker symbols
    """
    
    url = "https://companiesmarketcap.com/"
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    return [e.text for e in soup.select('div.company-name')]

In [14]:
len(scrape_names())

100

In [10]:
def scrape_symbols(num_pages: int) -> List:
    """Scrapes ticker symbols of top 100 US companies (based on market cap)
    Parameters
    ----------
    num_pages: int
        number of pages to scrape, each page has 100 symbols, max is 
    
    Returns
    -------
        list of strings: ticker symbols
    """
    list_tickers = []
    if num_pages == 1:
        url = 'https://companiesmarketcap.com/'
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'html.parser')
        list_tickers = [e.text for e in soup.select('div.company-code')]
    else: 
        for num_page in range(num_pages):
            url = f"https://companiesmarketcap.com/page/{num_page+1}/"
            html = requests.get(url).text
            soup = BeautifulSoup(html, 'html.parser')
            list_ticker_page = [e.text for e in soup.select('div.company-code')]
            list_tickers = list_tickers + list_ticker_page
    
    return list_tickers

In [12]:
scrape_symbols(num_pages=5)

['AAPL',
 'MSFT',
 '2222.SR',
 'GOOG',
 'AMZN',
 'TSLA',
 'FB',
 'BRK-A',
 'TSM',
 'NVDA',
 'TCEHY',
 'V',
 'UNH',
 'JNJ',
 'JPM',
 '005930.KS',
 'LVMUY',
 'PG',
 'WMT',
 '600519.SS',
 'BAC',
 'HD',
 'NSRGY',
 'MA',
 'BABA',
 'RHHBY',
 'XOM',
 'PFE',
 'ASML',
 'TM',
 'DIS',
 'KO',
 '1398.HK',
 'CVX',
 'CSCO',
 'ADBE',
 'PEP',
 'ABBV',
 'LLY',
 'TMO',
 'OR.PA',
 'CMCSA',
 'AVGO',
 'NKE',
 'RELIANCE.NS',
 'NFLX',
 'VZ',
 'ORCL',
 'ABT',
 'CRM',
 'NVO',
 'ACN',
 'COST',
 'WFC',
 'INTC',
 '300750.SZ',
 '3968.HK',
 'MRK',
 'PYPL',
 'DHR',
 'NVS',
 'T',
 'RYDAF',
 'TCS.NS',
 'CICHY',
 'MCD',
 'QCOM',
 'AZN',
 'MPNGF',
 'MS',
 'UPS',
 'PRX.VI',
 'SCHW',
 'BHP',
 'HESAF',
 'RY',
 'LIN',
 'NEE',
 'SAP',
 'ACGBY',
 'TXN',
 'PM',
 'UNP',
 'INTU',
 'PNGAY',
 'LOW',
 'TTE',
 'TD',
 'AMD',
 'PTR',
 'HON',
 'BMY',
 'CDI.PA',
 'MDT',
 'SONY',
 'HSBC',
 'AAIGF',
 'CVS',
 'SIEGY',
 '0941.HK',
 'RTX',
 'BACHF',
 'UL',
 'VOW3.DE',
 'SNY',
 '000858.SZ',
 'KYCCF',
 'TMUS',
 'AMGN',
 'SHOP',
 'HDB',
 'C',
 '

In [ ]:
import yfinance as yf
msft = yf.Ticker("MSFT")
msft.info["sharesOutstanding"]/1000000

7507.979776